We have seen that the LLM needs a complete picture of what it needs to do.

We have been adding extra data to the prompt and then running a new query.

Sometimes we need extra information that we can't provide at the start of the prompt. We can gived a list of tools to the LLM explaining what each does.

It can then select the right tool and arguments to get the extra data it needs. It will then send back the funtion and arguments that WE need to run, get the output and pass that into the next LLM request, building up a complete context picture for the LLM to answer.

We saw in ROUTER how we can get the LLM to select the most appropriate report or in this case tool.

When we say run it, the LLM passes 'us' information on the function that needs to be run and the arguments to be used. 

The LLM can also extract data from the prompt. We will see thuis more fullyu in `06_extraction_teachability.ipynb`

We then run the function and send back the output to the LLM by appending to the list of messages.


As we are seeing, the LLM request is stateless and can only be as effective as the instructions and data we provide it.

We can do this in a 'one time' request but if we don't know what to add in to the context, we can offer a set of tools and what they can offer to the LLM. 

It will then decide what it needs and let us know the functions and arguments WE need to run to then pass the output to the LLM to augment its context data.

In the cases where where an Agent calls a tool, the tool is called on our box.

Where are the functions run? From the OpenAI website, we can see at the arrow that tools are executed on our 'box'.

<img style="width:1100px;" src="./images/where-tools-are-executed.png">


Like routing, we can also ask the LLM to select the most appropriate tool to use.

In this example, we see how we can do Tool Selection and Data Extraction. The Agent determines what it will do next and also extracts data from the LLM response as arguments for the next step. 

In essence, the LLM decides on the tool/function it wants to use and send back the function name and arguments for us to run on our own box,
get the result and then add this to the messages in the next LLM request.


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import requests

In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-1WUVgv...
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


In [4]:
# original propmt
system_message = """
You are an assistant that is very good at determining what tool to use to solve queries.
"""

ai_programming = """

# TOOLS

You have two tools:

1. A calculator tool that can perform basic arithmetic operations, such as addition, subtraction, multiplication, and division. If this tool is used then respond in JSON FORMAT.

## Example JSON format:

{"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 5, "num2": 5, "operation": "addition"}} 

2. A jokes tool that can provide a light-hearted joke for the audience reqested. If this tool is used then respond in JSON FORMAT.

## Example JSON format:

{"tool": "joke", "next": "do_joke", "audience": "Pythonistas" }} 


NOTE: Only used tools if needed. If no tools are needed then respond in the usual way.

Thank you for your being accurate and complete with this query.
"""
system_message += ai_programming

In [5]:
user_prompt = "What is 120 plus 300?"
# user_prompt = "Tell me a joke when I am doing stand up at a Python Conference"
# user_prompt = "What is the capital of Ireland?"
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

In [6]:
response = client.chat.completions.create(model=MODEL, messages=prompts)
print(response.choices[0].message.content)
output = response.choices[0].message.content.replace("\n", "")

{"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 120, "num2": 300, "operation": "addition"}}


In [7]:
# As specified in the AI Programming section of the prompt, we get back a JSON response/dictionary with what
# the next step should be.

print(output)

{"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 120, "num2": 300, "operation": "addition"}}


now at END OF AI BIT

Having got the function and arguments, we extract these from the response and run it.

This is now regular Python rather than AI. We have called the 'AI as API' and now handle the response in the same way we might handle the response from a DB query.

I stress this as it is one of the personal outcomes - that Agentic Python is still traditional Python where our skills and experience are still needed.


In [8]:
try:
    json_output = output.strip()

    json_output = json.loads(json_output)

    print(json_output)

    # use do_next as 'next' is a reserved word

    do_next = json_output["next"]
except json.JSONDecodeError as e:
    print(f"No tool called so excepting...to be handled...")
    do_next = None

{'tool': 'calculator', 'next': 'do_calculation', 'arguments': {'num1': 120, 'num2': 300, 'operation': 'addition'}}


Now we use our day to day Python to run some code based on the next step, _very much like calling a Weather API and then writing code based on the result._

There are other ways we might code this but this will suffice for demonstration purposes.

In this demo, we use the calculator tool or joke tool to give a response. The tool is some code rather than a defined functuion, we will see this in the PLANNING demo `08` and `09`, where we will use the tools to call functions in our code which we then feedback to the LLM to carry out the next stage of the plan.


In [9]:
# this could be encapsulated into a function
if do_next == "do_joke":
    get_random_joke_internet = requests.get(
        "https://official-joke-api.appspot.com/random_joke"
    )
    print(get_random_joke_internet.json())

In [10]:
# this could be encapsulated into a function
# the main point is we have extracted the tool and data from the response
if do_next == "do_calculation":
    tool = json_output["tool"]
    num1 = json_output["arguments"]["num1"]
    num2 = json_output["arguments"]["num2"]
    operation = json_output["arguments"]["operation"]
    if operation == "addition":
        answer = num1 + num2
    elif operation == "subtraction":
        answer = num1 - num2
    elif operation == "multiplication":
        answer = num1 * num2
    elif operation == "division":
        answer = num1 / num2
    print(f"{num1} {operation} {num2} = {answer}")

120 addition 300 = 420


<img src="./images/05-tool.png" width=700px>